# Overture Maps Loader

`OvertureMapsLoader` can download the Overture Maps data from the s3 bucket for a given region.

It is a wrapper around the [`OvertureMaestro`](https://github.com/kraina-ai/overturemaestro) library that can download the data in the original format but also have some advanced functions.

In the `SRAI` context, `OvertureMapsLoader` utilizes so-called wide format for returning features with columns representing potential categories of the object.

In [ ]:
from shapely.geometry import box

from srai.constants import GEOMETRY_COLUMN
from srai.loaders import OvertureMapsLoader
from srai.regionalizers import geocode_to_region_gdf

## Using OvertureMapsLoader to download data for a specific area

### Download all available features in Paris, France

In [ ]:
loader = OvertureMapsLoader()
paris = geocode_to_region_gdf("Paris")
paris_features_gdf = loader.load(paris)
paris_features_gdf

### Plot features

Colours from the this palette: https://colorhunt.co/palette/f8ededff8225b43f3f173b45

In [ ]:
ax = paris.plot(color="#F8EDED", figsize=(16, 16))

# plot water
water_columns = [c for c in paris_features_gdf.columns if "water" in c]
water_data = paris_features_gdf[paris_features_gdf[water_columns].any(axis=1)]
# water_data.plot(ax=ax, color="#f0746e", markersize=0.1)
water_data.plot(ax=ax, color="#FF8225", markersize=0.1)

# plot_roads
roads_data = paris_features_gdf[paris_features_gdf["transportation|segment|road"]]
roads_data.plot(ax=ax, color="#B43F3F", markersize=0.1, linewidth=0.25)

# plot buildings
building_columns = [c for c in paris_features_gdf.columns if c.startswith("buildings")]
buildings_data = paris_features_gdf[paris_features_gdf[building_columns].any(axis=1)]
buildings_data.plot(ax=ax, color="#173B45", markersize=0.1)

xmin, ymin, xmax, ymax = paris.total_bounds
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)

ax.set_axis_off()

### Download more detailed data with higher hierarchy value

By default, the `hierarchy_depth` value is equal to `1`, but it can be set to `None` to get a full list of all possible columns.

In [ ]:
manhattan_bbox = box(-73.994551, 40.762396, -73.936872, 40.804239)
loader = OvertureMapsLoader(hierarchy_depth=None)
new_york_features_gdf = loader.load(manhattan_bbox)
new_york_features_gdf

As you can see, there are over `2600` columns available.

Let's see top 20 most popular columns. 

In [ ]:
new_york_features_gdf.drop(columns=GEOMETRY_COLUMN).sum().sort_values(ascending=False).head(20)